In [4]:
import os 
import re
import matplotlib.pyplot as plt
import rmgpy.chemkin
import numpy as np
import cantera as ct
import pandas as pd
%matplotlib inline



In [5]:
def load_chemkin_file(path): 
    """ 
    Load Chemkin file and extract the reactions and species.
    Path should be the path to the species folder.
    """
    
    full_path = os.path.join(path,'chemkin')
    chemkin_path = os.path.join(full_path,'copies', 'copy_chem_annotated.inp')
   # chemkin_path = os.path.join(full_path,'chem_annotated.inp')
    dictionary_path = os.path.join(full_path,'species_dictionary.txt')
    transport_path = os.path.join(full_path,'tran.dat')

    species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_path, dictionary_path=dictionary_path, transport_path=transport_path, check_duplicates=False)

    return species_list, reaction_list

In [72]:
############# load the chemkin file and the cantera dataframe ################

#file = 'sensitivites_copy_chem_annotated.cti.csv'
#species_list, reaction_list = load_chemkin_file('/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/methane_with_added_2_BTP/cantera/Nora/2_BTP'
#this is the same as below 

chemkin_path='/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/RMG_NK/copy_chem_annotated.inp'
dictionary_path='/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/RMG_NK/species_dictionary.txt'
transport_path='/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/RMG_NK/tran.dat'


species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_path, dictionary_path=dictionary_path, transport_path=transport_path,check_duplicates=False)

In [73]:
#we want to focus on the top ___ most sensitive reactions 
n = 20

#use the csv files in this directory
list_of_csv_files = [file for file in os.listdir('/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/sensitivity/subtract_sensitivity/RMG_original') if re.search('\.csv',file)]
'/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/sensitivity/subtract_sensitivity/RMG_original'
#also keep in mind that we already flipped the reactions of the following RMG reaction indices
already_flipped = [1, 746, 33, 30, 357, 374, 685, 58, 38, 91, 13]

## flipping kinetics

Using the subtract sensitivity of RMG at different 2-BTP volume fractions, lets find the most sensitive reactions in the 2-BTP NIST model and then change the kinetics of these reactions in RMG's 2_BTP model. 

In [78]:
def find_matches(csv_file): 
    
    df = pd.read_csv(f'/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/sensitivity/subtract_sensitivity/RMG_original/{csv_file}')
    
    RMG_rxn_match = []
    RMG_rxn_match_indices = []
    
    print(f'for csv file: {csv_file}')
    print('************************************************************************')
    #let's find a match for the Cantera reactions in sensitvity analysis and then flip them the RMG model to have NIST kinetics
    for index,row in df.head(n).iterrows(): #Only iterate over the first n rows
        

        i=0 # counter, for assertion error later 

        ct_index, sensitivity, ct_equation, ct_products, ct_reactants = row['cantera_index'], row['sensitivity'], row['cantera equation'], eval(row['cantera products']), eval(row['cantera reactants'])

        #list of all products in each reaction 
        list_of_products = [(k+',')*int(v) for k,v in ct_products.items()]   #use ',' as a marker for distinction between different species
        products = set(list(filter(None, (''.join(list_of_products)).split(',')))) #now make all the products into a list. If two of the same species are listed as reactants, they should appear as two entries

        #list of all reactants in each reaction 
        list_of_reactants = [(k+',')*int(v) for k,v in ct_reactants.items()]   #use ',' as a marker for distinction between different species
        reactants= set(list(filter(None, (''.join(list_of_reactants)).split(',')))) #now make all the products into a list. If two of the same species are listed as reactants, they should appear as two entries

        #print(products, reactants)
        #iterate through rmg reactions and find the match to the cantera reaction
        for index, rmg_rxn in enumerate(reaction_list): 
            
                try: 
                    rmg_reactants = set([str(i) for i in rmg_rxn.reactants])
                    rmg_products = set([str(i) for i in rmg_rxn.products])

                    #to correct a difference in labeling between RMG and NIST. CH2(T) and CH2* are the same 
                    if ('CH2*' in reactants) or ('CH2*' in products):  
                        if 'CH2(T)' in rmg_reactants:
                            rmg_reactants.remove('CH2(T)') 
                            rmg_reactants.add('CH2*')
                        if 'CH2(T)'in rmg_products:
                            rmg_products.remove('CH2(T)') 
                            rmg_products.add('CH2*')

                    if (reactants==rmg_reactants) and (products==rmg_products):
                        rmg_rxn_to_be_changed=rmg_rxn
                        print(rmg_rxn)
                        print(ct_equation)
                        print(rmg_rxn.index)
                        RMG_rxn_match.append(rmg_rxn)
                        RMG_rxn_match_indices.append(rmg_rxn.index)
                        print('***************************************************************')
                        i += 1 #to check if match was found 
                        break

                    #if equation is written backwards 
                    if (reactants==rmg_products) and (products==rmg_reactants):
                        rmg_rxn_to_be_changed=rmg_rxn
                        print(rmg_rxn)
                        print(ct_equation)
                        print(f'backwards, {rmg_rxn.index}')
                        RMG_rxn_match.append(rmg_rxn)
                        RMG_rxn_match_indices.append(rmg_rxn.index)
                        print('**************************************')
                        i += 1 #to check if match was found 
                        break

                except AttributeError: #some of the rmg_rxn.to_cantera() is a list of rxns
                    print(rmg_rxn)
                    for rxn in rmg_rxn: 
                        rmg_reactants = set([str(i) for i in rxn.reactants])
                        rmg_products = set([str(i) for i in rxn.products])

                        if (reactants==rmg_reactants) and (products==rmg_products):
                            rmg_rxn_to_be_changed=rmg_rxn
                            print(rmg_rxn.index)
                            RMG_rxn_match.append(rmg_rxn)
                            RMG_rxn_match_indices.append(rmg_rxn.index)
                            i += 1 #to check if match was found 
                            break

        
        #if you can't find a match, tell me that 
        if i!=1:
            print(f"Could not find match for NIST reaction: {ct_equation}")
            print('**************************************')
        
    return RMG_rxn_match, RMG_rxn_match_indices
        #if you can find a match, make a new chemkin file with the RMG kinetics replaced by the NIST kinetics. 
        #NOTE: NOT PLAYING WITH THERMO

        #feel like this script will take too long, so will just change chemkins by hand. 






In [85]:
##### execute 

RMG_indices_not_matched_yet = []
RMG_rxns_not_matched_yet = []

for file in list_of_csv_files: 
    RMG_rxn_match, RMG_rxn_match_indices = find_matches(file)
    for rxn_index in RMG_rxn_match_indices: 
        if (rxn_index not in already_flipped) and (rxn_index not in RMG_indices_not_matched_yet): 
            RMG_indices_not_matched_yet.append(rxn_index)
            for rxn in RMG_rxn_match:
                if rxn.index==rxn_index:
                    RMG_rxns_not_matched_yet.append(rxn)

for csv file: chem_annotated.cti_0.018075555555555553_sensitivities.csv
************************************************************************
BR(90) + HCO(17) <=> HBR(92) + CO(15)
BR(90) + HCO(17) <=> CO(15) + HBR(92)
746
***************************************************************
O2(4) + H(8) <=> O(9) + OH(2)
H(8) + O2(4) <=> O(9) + OH(2)
1
***************************************************************
OH(2) + CO(15) <=> H(8) + CO2(16)
CO(15) + OH(2) <=> CO2(16) + H(8)
30
***************************************************************
O2(4) + H(8) <=> HO2(13)
H(8) + O2(4) <=> HO2(13)
357
***************************************************************
H(8) + HBR(92) <=> BR(90) + H2(10)
H(8) + HBR(92) <=> BR(90) + H2(10)
685
***************************************************************
OH(2) + HCO(17) <=> H2O(5) + CO(15)
HCO(17) + OH(2) <=> CO(15) + H2O(5)
37
***************************************************************
H(8) + HCO(17) <=> H2(10) + CO(15)
H(8) + HCO(17) <=> C

In [90]:
RMG_indices_not_matched_yet

[37,
 34,
 412,
 15,
 97,
 16,
 763,
 94,
 4,
 124,
 123,
 104,
 163,
 117,
 411,
 1030,
 245,
 558,
 663]

In [89]:
for i in RMG_rxns_not_matched_yet:
    print(i.index)
#great that this cell and the one above match 

37
34
412
15
97
16
763
94
4
124
123
104
163
117
411
1030
245
558
663


Now lets find if there are any matching NIST reactions to the RMG reactions that haven't been matched yet

In [97]:
def compare_rxns(model_1_reactions, model_2_reactions): 
    '''
    Compares the reaction equations of each model. Uses to_cantera() on RMG model to eliminate chemkin_identifier 
    '''

################### mostly taken from diffmodel.py, compare_model_reactions() ###################

    #remove reactions with unknown species
    to_remove = []
    for reactionList in (model_1_reactions, model_2_reactions):
        for reaction in reactionList:
            for side in (reaction.products, reaction.reactants):
                for species in side:
                    if not species.molecule:
                        to_remove.append((reactionList, reaction))
                        print("Removing reaction {!r} that had unidentified species {!r}".format(reaction, species))
                        break
    if len(to_remove)!=0:
        for reactionList, reaction in to_remove:
            reactionList.remove(reaction)
        print(f'Following reactions removed because they have unknown species: {to_remove}')
    
    #find the common reactions and the unique reactions
    common_reactions = []
    common_reactions_equations = []

    copy_of_model_1_reactions = model_1_reactions.copy() #list 1
    copy_of_model_2_reactions = model_2_reactions.copy() #list 2

    
    for model_2_rxn in copy_of_model_2_reactions:
        for model_1_rxn in copy_of_model_1_reactions: # make a copy so you don't remove from the list you are iterating over
            counter = 0 #use this to see how many times this loop below was executed
            if model_2_rxn.is_isomorphic(model_1_rxn):
                common_reactions.append([model_1_rxn, model_2_rxn])
                common_reactions_equations.append([str(model_1_rxn), str(model_2_rxn)])
                copy_of_model_1_reactions.remove(model_1_rxn)
                counter += 1
            if counter>1:
                print([str(model_1_rxn),str(model_2_rxn)]) #let's see if a rxn has two matches
                
 
   
    
    #find the reactions that have different kinetics (aren't identical and aren't similar)
    
    different_kinetics_reactions = []
    
    for r1, r2 in common_reactions: 
        if (r2.kinetics.is_identical_to(r1.kinetics)==False) and (r2.kinetics.is_similar_to(r1.kinetics)==False):
            different_kinetics_reactions.append((r1, r2))
            
    return different_kinetics_reactions, common_reactions, common_reactions_equations
    
    

In [98]:
#we will use the NIST chemkin this time! 
chemkin_path='/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/NIST/2-BTP_kinetics_with_M.inp'
dictionary_path='/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/NIST/species_dictionary-2_BTP.txt'


NIST_species_list, NIST_reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin_path, dictionary_path=dictionary_path, check_duplicates=False)



different_kinetics_reactions, common_reactions, common_reactions_equations = compare_rxns(RMG_rxns_not_matched_yet, NIST_reaction_list)

In [103]:
common_reactions_equations

[['OH(2) + H2(10) <=> H(8) + H2O(5)', 'H2 + OH <=> H + H2O'],
 ['H(8) + HO2(13) <=> O2(4) + H2(10)', 'H2 + O2 <=> H + HO2'],
 ['H(8) + HO2(13) <=> OH(2) + OH(2)', 'H + HO2 <=> OH + OH'],
 ['H(8) + HCO(17) <=> H2(10) + CO(15)', 'H + HCO <=> CO + H2'],
 ['OH(2) + HCO(17) <=> H2O(5) + CO(15)', 'HCO + OH <=> CO + H2O'],
 ['H(8) + CH3(19) <=> CH4(3)', 'CH3 + H <=> CH4'],
 ['OH(2) + CH3(19) <=> H2O(5) + CH2(S)(25)', 'CH3 + OH <=> CH2* + H2O'],
 ['HO2(13) + CH3(19) <=> OH(2) + CH3O(27)', 'CH3 + HO2 <=> CH3O + OH'],
 ['CH3(19) + CH3(19) <=> H(8) + C2H5(32)', 'CH3 + CH3 <=> C2H5 + H'],
 ['H(8) + CH2OH(33) <=> OH(2) + CH3(19)', 'CH2OH + H <=> CH3 + OH'],
 ['H(8) + CH4(3) <=> H2(10) + CH3(19)', 'CH4 + H <=> CH3 + H2'],
 ['O(9) + CH4(3) <=> OH(2) + CH3(19)', 'CH4 + O <=> CH3 + OH'],
 ['O(9) + C2H2(23) <=> H(8) + HCCO(21)', 'C2H2 + O <=> H + HCCO'],
 ['O2(4) + CF2(43) <=> O(9) + CF2O(49)', 'CF2 + O2 <=> CF2:O + O'],
 ['BR(90) + H(8) <=> HBR(92)', 'BR + H <=> HBR'],
 ['BR(90) + HO2(13) <=> O2(4) + H

In [104]:
RMG_indices_not_matched_yet #this is not in the same order as above

[37,
 34,
 412,
 15,
 97,
 16,
 763,
 94,
 4,
 124,
 123,
 104,
 163,
 117,
 411,
 1030,
 245,
 558,
 663]

In [6]:
import os 
import re

s = [file for file in os.listdir('/work/westgroup/nora/Code/projects/Burgess_Comments/2_BTP_optimization/models/edited_RMG_subtracted_sensitivities') if re.search('\.cti',file)]

In [7]:
' '.join(s)

'rxn_1030.cti rxn_104.cti rxn_117.cti rxn_123.cti rxn_124.cti rxn_15.cti rxn_16.cti rxn_163.cti rxn_245.cti rxn_34.cti rxn_37.cti rxn_4.cti rxn_411.cti rxn_412.cti rxn_558.cti rxn_663.cti rxn_763.cti rxn_94.cti rxn_97.cti'